In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression, chi2, SelectPercentile, mutual_info_regression
from sklearn.ensemble import IsolationForest, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.neighbors import LocalOutlierFactor
import math
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score

In [2]:
SEED = 4426008 #np.random.randint(2**32)
np.random.seed(SEED)

In [3]:
just_scoring = False
folder = ""
if just_scoring:
  folder = folder + "just_scoring/"
else:
  folder = folder + "testing/"
raw = folder + "feature_reduced/" #"results/"
preprocessed = folder + "results/"
super_raw = folder + "feature_reduced2/"

In [4]:
import os
os.path.isdir(raw)

True

In [5]:
x_train = pd.read_csv(raw + 'X_train_preprocessed.csv',index_col=['id'])
x_test = pd.read_csv(raw + 'X_test_preprocessed.csv',index_col=['id'])
y_train = pd.read_csv(raw + 'y_train_preprocessed.csv',index_col=['id'])
y_test = pd.read_csv(raw + 'y_test_preprocessed.csv',index_col=['id'])

x_train_etr = pd.read_csv(super_raw + 'X_train_etr_preprocessed.csv',index_col=['id'])
x_test_etr = pd.read_csv(super_raw + 'X_test_etr_preprocessed.csv',index_col=['id'])

x_train

,x2,x6,x8,x17,x23,x42,x51,x60,x65,x69,...,x735,x742,x754,x774,x776,x782,x783,x795,x827,x828
id,,,,,,,,,,,,,,,,,,,,,
124.0,-0.640153,1.413600,-0.458523,0.727484,-0.407202,-0.360423,-0.801644,-0.773290,1.065081,-0.537347,...,0.517242,-1.223790,0.536230,-0.843395,-0.795930,0.686460,-0.424909,0.531791,-1.674551,-1.009797
178.0,0.554127,1.363075,0.026355,0.025575,0.447635,1.141528,-0.852888,0.155695,-1.284154,0.698667,...,-0.351589,0.074811,-0.000538,0.114315,0.863238,-0.008944,1.087813,-0.242313,0.626327,-0.701236
1088.0,0.155643,-0.868452,-0.036660,-0.269913,0.516866,-0.804754,-0.057434,1.065110,-0.492981,0.536688,...,-0.326767,0.218743,-0.794325,-0.341438,0.304049,-0.744808,-0.560412,-0.001115,0.368863,-0.051142
15.0,0.201366,0.041130,-0.739757,-0.103427,-0.220532,-0.838797,-0.204203,-0.842002,0.284307,0.340760,...,0.002186,0.250288,0.731188,-0.319969,-0.854192,-0.028950,1.085784,-0.019192,0.740875,-0.021295
1058.0,-0.931668,0.484016,-0.919151,-0.178743,-1.577617,-0.896364,-0.576772,0.318287,-0.851589,-0.827132,...,0.496186,-1.105829,0.440123,-0.692422,0.333676,-0.429249,-1.323298,0.477411,-0.782385,0.154279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354.0,-0.476903,2.015711,-1.029994,-0.579864,0.577311,0.955033,0.855709,0.279687,-0.131841,-0.065176,...,0.980969,-0.487768,-0.714850,0.155130,0.773954,0.195495,0.602806,-0.253744,0.222565,1.011351
1103.0,2.508357,-0.063506,-0.071101,-0.061963,1.895425,-0.216826,-0.162210,-0.056163,0.083830,3.511215,...,-0.015202,5.345413,-0.045588,2.421990,-0.052652,-0.060759,1.979639,0.017578,2.340883,0.079844
756.0,-0.021369,0.062090,0.860957,-1.380272,-0.187886,-0.584701,-1.404156,0.548543,0.012168,0.213590,...,0.777336,-0.291067,0.900660,-1.188400,-0.536699,0.619853,0.238338,-0.560214,-0.446793,-0.847286


In [19]:
x_train_etr

,x92,x115,x133,x159,x194,x200,x209,x242,x306,x309,...,x548,x640,x641,x642,x654,x702,x720,x726,x742,x828
id,,,,,,,,,,,,,,,,,,,,,
0.0,0.258960,2.652717,-0.244960,1.257218,0.072375,0.603809,-0.163344,0.462106,0.265865,0.122481,...,-0.139137,0.155596,0.810293,0.292890,0.382583,1.119060,-0.325089,-0.126788,-0.215107,-0.116709
1.0,-0.648939,-0.258528,0.713896,-0.526655,0.022061,-0.008352,0.848784,0.630885,0.151983,0.703157,...,0.650773,1.666427,-0.587788,-0.063254,-0.041439,-0.153122,2.128399,1.977946,16.382172,-1.420287
2.0,0.532833,-0.192500,0.049381,-0.020296,0.326692,-0.236306,-0.626414,-0.699048,0.246257,0.119217,...,-0.190728,1.191706,-0.255010,0.835397,0.137379,-0.176823,-0.205927,-0.954179,-0.723249,0.373456
3.0,0.222607,2.553724,-2.513515,1.006787,-2.132048,-1.892131,-1.774690,-2.499902,-1.673594,-1.712223,...,-0.542104,-1.380458,1.053739,0.778632,-1.810273,0.911097,-1.117881,-2.159832,-2.379375,0.013729
4.0,0.555736,0.946484,0.191480,-0.388803,0.210041,-0.740283,-0.313150,1.093712,0.141881,0.242958,...,-0.086018,0.942075,0.744850,0.344749,-0.406522,0.924617,0.622829,0.976201,0.128821,-0.081002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,0.058909,-0.238549,0.552295,-0.296473,0.816534,0.441544,0.886290,-0.225183,0.529080,0.521040,...,0.672576,0.182020,0.022773,-0.139579,0.354356,0.000145,-0.332122,0.405404,2.024306,1.133674
1208.0,0.135193,0.335474,-0.177007,-0.388619,-0.112061,-0.736898,-0.710501,0.490008,-0.652086,-1.044285,...,-0.051786,-0.581512,1.565244,0.237432,-0.620634,0.084754,1.187857,1.352225,0.414920,0.668279
1209.0,-0.408872,-0.121320,0.311670,-0.286656,0.377444,0.991590,0.154650,-0.356193,0.754307,1.113552,...,0.597577,0.591914,0.158754,1.113664,0.819885,-0.443699,-0.181584,0.182359,12.783911,0.488665


In [6]:
TRAIN_RATIO = 0.8

def train_len(data = None):
    if data is None:
        data = x_train
    return math.floor(TRAIN_RATIO*len(data))

def validation_len(data = None):
    if data is None:
        data = x_train
    return len(data) - train_len(data)

def train(data = None):
    if data is None:
        data = x_train
    if isinstance(data, np.ndarray):
        return data[:train_len(data)].copy()
    return data.iloc[:train_len(data)].copy()

def validation(data = None):
    if data is None:
        data = x_train
    if isinstance(data, np.ndarray):
        return data[-validation_len(data):].copy()
    return data.iloc[-validation_len(data):].copy()

def full_X(x_tr = None, x_ts = None):
    if x_tr is None:
        x_tr = x_train
    if x_ts is None:
        x_ts = x_test
    return pd.concat([x_tr, x_ts], join = "inner")

def performance(y_pred):
    if not y_test.empty:
        return r2_score(y_test, y_pred)

In [7]:
kernel = RationalQuadratic()
gpr = GaussianProcessRegressor(kernel = kernel, normalize_y = True)
gpr.fit(x_train, np.ravel(y_train))
y_pred_gpr = gpr.predict(x_test)
print(performance(y_pred_gpr))
#0.5174025667796436

None


In [8]:
etr = ExtraTreesRegressor(n_estimators = 5000, n_jobs=-1)
etr.fit(x_train, np.ravel(y_train))
y_pred_etr = etr.predict(x_test)
print(performance(y_pred_etr))

None


In [9]:
etr2 = ExtraTreesRegressor(n_estimators = 5000, n_jobs=-1)
etr2.fit(x_train_etr, np.ravel(y_train))
y_pred_etr2 = etr2.predict(x_test_etr)
print(performance(y_pred_etr2))

None


In [10]:
# from sklearn.preprocessing import StandardScaler
# s = StandardScaler()
# s.fit(np.array(pd.concat([x_train_etr, x_test_etr], join = "inner")))
# x_train_etr[[col for col in x_train_etr.columns]] = s.transform(np.array(x_train_etr))
# x_test_etr[[col for col in x_train_etr.columns]] = s.transform(np.array(x_test_etr))

In [11]:
kernel = RationalQuadratic()
gpr2 = GaussianProcessRegressor(kernel = kernel, normalize_y = True)
gpr2.fit(x_train_etr, np.ravel(y_train))
y_pred_gpr2 = gpr2.predict(x_test_etr)
print(performance(y_pred_gpr2))
# #0.5174025667796436

None


C:\Users\david\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\david\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [23]:
gbr = GradientBoostingRegressor(
    n_estimators=80
)
gbr.fit(np.array(x_train_etr), np.ravel(y_train))
y_pred_gbr = gbr.predict(np.array(x_test_etr))

In [26]:
%%time
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
level0 = list()
# level0.append(('GP', gpr))
# level0.append(('extra_tree', etr))
level0.append(('GP2', gpr2))
level0.append(('extra_tree2', etr2))
level0.append(('gbr2', gbr))
level1 = LinearRegression()
stacking_reg_4 = StackingRegressor(estimators=level0, final_estimator=level1)
stacking_reg_4.fit(np.array(x_train), np.ravel(y_train))

y_stacking_3 = stacking_reg_4.predict(np.array(x_test))
print(performance(y_stacking_3))

if not just_scoring:
  output_df4 = pd.DataFrame(y_stacking_3, columns = ['y'])
  output_df4.to_csv(preprocessed + 'output4.csv', index_label="id")
  output_df4.to_csv(preprocessed + 'output4.txt', index_label="id")
  print(output_df4)

None
             y
0    58.488743
1    77.289569
2    72.196318
3    76.120101
4    71.960218
..         ...
771  65.928961
772  74.988414
773  75.474368
774  67.674304
775  59.302553

[776 rows x 1 columns]
CPU times: total: 3min 37s
Wall time: 1min 20s


In [24]:
output_df = pd.DataFrame(y_pred_gbr, columns = ['y'])
output_df

,y
0,70.037483
1,67.360606
2,69.453297
3,65.675555
4,70.272935
...,...
771,70.534239
772,68.708907
773,69.597840
774,70.334794


In [25]:
output_df.to_csv(preprocessed + 'output.csv', index_label="id")
output_df.to_csv(preprocessed + 'output.txt', index_label="id")

In [15]:
gpr.kernel_

RationalQuadratic(alpha=0.338, length_scale=3.93)